In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

In [28]:
train=pd.read_csv('D:/../iris_training.csv')

In [29]:
test=pd.read_csv('D:/../iris_test.csv')

In [30]:
train.head()

,120,4,setosa,versicolor,virginica
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [31]:
X_train = train.iloc[:,0:4]
y_train =train.iloc[:,4]

In [32]:
n_inputs = len(X_train.iloc[0,:])
n_hidden1 = 300 #number of neurons in first layer
n_hidden2 = 100 #number of neurons in second layer
n_outputs = len(np.unique(y_train))

In [33]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y") 

In [37]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu, reuse=True)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu, reuse=True)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs", reuse=True)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('log_loss', loss)
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [38]:
file_writer = tf.summary.FileWriter('D:/Hands_on_ML_Assignments/Self/Logdir', tf.get_default_graph())

In [39]:
X_valid = test.iloc[:,0:4]
y_valid = test.iloc[:,4]

In [59]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    data = np.array(data)
    idx = np.arange(0 , len(data))
   
    np.random.shuffle(idx)
    idx = idx[:num]
    #print("new")
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [62]:
m, n = X_train.shape
n_epochs = 10001
batch_size = 50
n_batches = int(np.ceil(m / batch_size))

checkpoint_path = "/tmp/ten.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./ten"

best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress = 50

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        # if the checkpoint file exists, restore the model and load the epoch number
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)

    for epoch in range(start_epoch, n_epochs):
        for iteration in range(len(train) // batch_size):
            X_batch, y_batch = next_batch(batch_size, X_train, y_train)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val, loss_val, accuracy_summary_str, loss_summary_str = sess.run([accuracy, loss, accuracy_summary, loss_summary], feed_dict={X: X_valid, y: y_valid})
        file_writer.add_summary(accuracy_summary_str, epoch)
        file_writer.add_summary(loss_summary_str, epoch)
        if epoch % 5 == 0:
            print("Epoch:", epoch,
                  "\tValidation accuracy: {:.3f}%".format(accuracy_val * 100),
                  "\tLoss: {:.5f}".format(loss_val))
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epoch + 1))
            if loss_val < best_loss:
                saver.save(sess, final_model_path)
                best_loss = loss_val
            else:
                epochs_without_progress += 5
                if epochs_without_progress > max_epochs_without_progress:
                    print("Early stopping")
                    break

Epoch: 0 	Validation accuracy: 26.667% 	Loss: 1.07763
Epoch: 5 	Validation accuracy: 53.333% 	Loss: 0.90381
Epoch: 10 	Validation accuracy: 56.667% 	Loss: 0.82238
Epoch: 15 	Validation accuracy: 76.667% 	Loss: 0.73836
Epoch: 20 	Validation accuracy: 56.667% 	Loss: 0.70373
Epoch: 25 	Validation accuracy: 63.333% 	Loss: 0.65197
Epoch: 30 	Validation accuracy: 70.000% 	Loss: 0.61492
Epoch: 35 	Validation accuracy: 73.333% 	Loss: 0.58499
Epoch: 40 	Validation accuracy: 93.333% 	Loss: 0.53872
Epoch: 45 	Validation accuracy: 80.000% 	Loss: 0.53277
Epoch: 50 	Validation accuracy: 86.667% 	Loss: 0.50993
Epoch: 55 	Validation accuracy: 83.333% 	Loss: 0.49582
Epoch: 60 	Validation accuracy: 80.000% 	Loss: 0.49768
Epoch: 65 	Validation accuracy: 83.333% 	Loss: 0.47320
Epoch: 70 	Validation accuracy: 83.333% 	Loss: 0.45808
Epoch: 75 	Validation accuracy: 90.000% 	Loss: 0.43623
Epoch: 80 	Validation accuracy: 80.000% 	Loss: 0.44685
Epoch: 85 	Validation accuracy: 93.333% 	Loss: 0.39740
Epoch: 90 	V

In [57]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [63]:
show_graph(tf.get_default_graph())